# NCF with Focal Loss - ML-100K Improved Experimental Design

**Paper**: "Addressing Class Imbalance in NCF with Focal Loss" (AAMAS 2025)

## Objective

This notebook implements improved experimental methodology to test three key hypotheses:

**H1 (Efficacy)**: Focal Loss improves NeuMF performance over standard BCE loss on implicit feedback recommendation.

**H2 (Robustness)**: The improvement is robust across different negative sampling ratios (1:4, 1:10, 1:50).

**H3 (Mechanism)**: The focusing effect (gamma > 0) is necessary beyond simple class weighting (alpha-balanced BCE).

## Improvements Over Original Design

1. **Primary experiment at 1:10 sampling** (not just 1:4) - more realistic for real systems
2. **Alpha-balanced BCE control** - isolates focusing effect from class weighting
3. **Alpha-sampling interaction analysis** - addresses confound between alpha and sampling ratio
4. **Robustness study** - tests across 3 sampling ratios (1:4, 1:10, 1:50)
5. **Training dynamics tracking** - validates mechanism by analyzing loss contribution by confidence bin
6. **Proper statistical framing** - clear hypothesis testing structure

## Dataset: MovieLens 100K

- 100,000 ratings from 943 users on 1,682 movies
- Binarization: ratings >= 4 -> positive interaction
- Leave-one-out evaluation (most recent for test)
- Full ranking evaluation

## Cell 1: Suppress Warnings

In [1]:
# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

## Cell 2: Install Dependencies

**Instructions:**
1. Run the install cell below
2. **RESTART** the runtime (Runtime -> Restart session)
3. Run the verification cell
4. Continue with remaining cells

In [2]:
# ============================================
# Install Dependencies (Part 1)
# ============================================
# After running this cell, RESTART the runtime, then run the next cell

%pip install -q ray
%pip install -q recbole==1.2.0
%pip install -q kmeans-pytorch

# Force numpy 1.x (required for RecBole compatibility)
%pip uninstall -y numpy
%pip install -q "numpy<2"

print("\n" + "="*60)
print("RESTART REQUIRED")
print("="*60)
print("Go to: Runtime -> Restart session")
print("Then run the NEXT cell to verify installation.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.0 MB/s eta 0:00:00
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 129.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.36.3 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tobler 0.13.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires

In [1]:
# ============================================
# Verify Installation (Part 2) - Run AFTER restart
# ============================================
import numpy as np
print(f"NumPy version: {np.__version__}")

if np.__version__.startswith("2."):
    print("\nERROR: NumPy 2.x still detected!")
    print("Try: Runtime -> Restart session -> Run this cell again")
else:
    print("SUCCESS: NumPy 1.x installed. Continue to next cell.")

NumPy version: 1.26.4
SUCCESS: NumPy 1.x installed. Continue to next cell.


## Cell 3: Imports

In [2]:
# Standard imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
import numpy as np
import pandas as pd
from collections import defaultdict
import os
import logging
import sys

# ============================================
# Environment Setup (Colab / Local)
# ============================================
if 'google.colab' in sys.modules:
    # Clone repo if not already present
    if not os.path.exists('/content/Project-2026---Team-7'):
        !git clone https://github.com/omereliy/Project-2026---Team-7.git /content/Project-2026---Team-7
    # Add experiments folder to path
    sys.path.insert(0, '/content/Project-2026---Team-7/experiments')
    %cd /content/Project-2026---Team-7/experiments
    print("Running on Google Colab - repo cloned")
else:
    # Local: assume running from experiments folder
    sys.path.insert(0, '.')
    print("Running locally")

# ============================================
# RecBole Fix: Patch torch.distributed.barrier for single-GPU
# See: https://github.com/RUCAIBox/RecBole/issues/1989
# ============================================
if not hasattr(dist, '_barrier_patched'):
    _original_barrier = dist.barrier
    def _patched_barrier(*args, **kwargs):
        if dist.is_available() and dist.is_initialized():
            return _original_barrier(*args, **kwargs)
        # Skip barrier if not in distributed mode
    dist.barrier = _patched_barrier
    dist._barrier_patched = True
    print("Applied RecBole distributed fix")

# PyTorch 2.6+ compatibility patch
if not hasattr(torch, '_load_patched'):
    _original_torch_load = torch.load
    def _patched_torch_load(*args, **kwargs):
        if 'weights_only' not in kwargs:
            kwargs['weights_only'] = False
        return _original_torch_load(*args, **kwargs)
    torch.load = _patched_torch_load
    torch._load_patched = True

from focal_loss_utils import (
    # Loss functions
    FocalLoss, AlphaBalancedBCE,
    # Configuration
    get_base_config, get_neumf_config,
    # Training functions
    train_neumf_focal_loss, train_neumf_alpha_bce,
    # Evaluation
    create_comparison_table, compute_improvement,
    # Validation
    validate_focal_loss_implementation, demonstrate_focal_loss_effect,
    # Alpha-sampling analysis
    analyze_alpha_sampling_interaction, compute_effective_class_ratio,
    get_balanced_alpha,
    # Multi-seed experiments
    run_multi_seed_experiment
)

# RecBole imports
from recbole.quick_start import run_recbole
from recbole.model.general_recommender.neumf import NeuMF
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")

Cloning into '/content/Project-2026---Team-7'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 211 (delta 117), reused 152 (delta 75), pack-reused 0 (from 0)
Receiving objects: 100% (211/211), 436.08 KiB | 6.51 MiB/s, done.
Resolving deltas: 100% (117/117), done.
/content/Project-2026---Team-7/experiments
Running on Google Colab - repo cloned
Applied RecBole distributed fix
Using GPU: NVIDIA A100-SXM4-40GB


## Cell 4: Focal Loss Validation Test

Verify that the Focal Loss implementation is correct:
1. FL(gamma=0, alpha=0.5) = 0.5 * BCE
2. Higher gamma reduces loss for well-classified examples

In [3]:
print("="*70)
print("VALIDATION: Focal Loss Implementation")
print("="*70)

# Test implementation correctness
validate_focal_loss_implementation()

print("\n" + "="*70)
print("DEMONSTRATION: Focal Loss Effect on Easy vs Hard Examples")
print("="*70)

# Demonstrate focusing effect
demo_df = demonstrate_focal_loss_effect()
print("\n" + demo_df.to_string(index=False))

print("\nInterpretation:")
print("- Easy examples (high confidence) are down-weighted by 10-100x")
print("- Hard examples (low confidence) retain most of their loss contribution")
print("- This forces the model to focus on hard-to-classify examples")

VALIDATION: Focal Loss Implementation
Focal Loss implementation PASSED all tests!

DEMONSTRATION: Focal Loss Effect on Easy vs Hard Examples

                      scenario  bce_loss  focal_loss  reduction_factor
Easy negative (pred=0.05, y=0)  0.051293    0.000096        533.332988
 Hard positive (pred=0.3, y=1)  1.203973    0.147487          8.163266
 Hard negative (pred=0.7, y=0)  1.203973    0.442460          2.721089
Easy positive (pred=0.95, y=1)  0.051293    0.000032       1599.999252

Interpretation:
- Easy examples (high confidence) are down-weighted by 10-100x
- Hard examples (low confidence) retain most of their loss contribution
- This forces the model to focus on hard-to-classify examples


## Cell 5: Experiment Configuration

We test three negative sampling ratios:
- **1:4** - Standard ratio used in many NCF papers
- **1:10** - **PRIMARY EXPERIMENT** - more realistic for production systems
- **1:50** - High negative ratio to test robustness under extreme imbalance

The primary experiment uses 1:10 sampling because:
1. It's more representative of real recommendation systems
2. It provides stronger evidence of robustness than just testing at 1:4
3. The class imbalance problem is more pronounced, making Focal Loss improvements more meaningful

In [4]:
# Dataset and sampling ratios
DATASET = 'ml-100k'
SAMPLING_RATIOS = [4, 10, 50]  # negative samples per positive
PRIMARY_RATIO = 10  # Main experiment uses 1:10 sampling

# Focal Loss hyperparameters (from literature)
GAMMA = 2.0  # Focusing parameter
ALPHA = 0.25  # Class balancing weight for positives

print(f"Dataset: {DATASET}")
print(f"Negative sampling ratios: {SAMPLING_RATIOS}")
print(f"Primary experiment: 1:{PRIMARY_RATIO} sampling")
print(f"\nFocal Loss hyperparameters:")
print(f"  gamma = {GAMMA} (focusing parameter)")
print(f"  alpha = {ALPHA} (class balancing weight for positives)")

Dataset: ml-100k
Negative sampling ratios: [4, 10, 50]
Primary experiment: 1:10 sampling

Focal Loss hyperparameters:
  gamma = 2.0 (focusing parameter)
  alpha = 0.25 (class balancing weight for positives)


## Cell 6: Configuration Setup

Create configurations for each sampling ratio

In [5]:
# Create configurations for each sampling ratio
configs = {}
for ratio in SAMPLING_RATIOS:
    base = get_base_config(DATASET, device, neg_sample_num=ratio)
    configs[ratio] = get_neumf_config(base)
    print(f"Configuration for 1:{ratio} sampling created")

print("\nNeuMF Architecture:")
print(f"  MF Embedding Size: {configs[PRIMARY_RATIO]['mf_embedding_size']}")
print(f"  MLP Embedding Size: {configs[PRIMARY_RATIO]['mlp_embedding_size']}")
print(f"  MLP Hidden Layers: {configs[PRIMARY_RATIO]['mlp_hidden_size']}")
print(f"  Dropout: {configs[PRIMARY_RATIO]['dropout_prob']}")
print(f"\nTraining Settings:")
print(f"  Max Epochs: {configs[PRIMARY_RATIO]['epochs']}")
print(f"  Early Stopping Patience: {configs[PRIMARY_RATIO]['stopping_step']}")
print(f"  Learning Rate: {configs[PRIMARY_RATIO]['learning_rate']}")
print(f"  Batch Size: {configs[PRIMARY_RATIO]['train_batch_size']}")

Configuration for 1:4 sampling created
Configuration for 1:10 sampling created
Configuration for 1:50 sampling created

NeuMF Architecture:
  MF Embedding Size: 64
  MLP Embedding Size: 64
  MLP Hidden Layers: [128, 64, 32]
  Dropout: 0.0

Training Settings:
  Max Epochs: 100
  Early Stopping Patience: 10
  Learning Rate: 0.001
  Batch Size: 256


## Cell 6b: Load Saved Results (Optional)

**Skip training and load pre-computed results.**

If you want to skip re-running experiments (e.g., for analysis or after runtime restart), run this cell to load saved results from `saved_results_ml100k.py`.

Set `LOAD_SAVED_RESULTS = True` to use saved results, or `False` to train from scratch.

In [7]:
# ============================================
# LOAD SAVED RESULTS (Optional)
# ============================================
# Set to True to skip training and use pre-computed results
# Set to False to train models from scratch

LOAD_SAVED_RESULTS = False  # <-- Change this to False to train from scratch

if LOAD_SAVED_RESULTS:
    from saved_results_ml100k import (
        load_all_results, print_summary,
        RESULT_BCE, RESULT_ALPHA_BCE, RESULT_FOCAL,
        ROBUSTNESS_RESULTS, GRID_RESULTS
    )

    # Load all results
    saved = load_all_results()

    # Set variables to match notebook expectations
    result_bce = saved['result_bce']
    result_alpha_bce = saved['result_alpha_bce']
    result_focal = saved['result_focal']
    robustness_results = saved['robustness_results']
    grid_results = saved['grid_results']

    # Print summary
    print_summary()

    print("\nVariables loaded:")
    print("  - result_bce")
    print("  - result_alpha_bce")
    print("  - result_focal")
    print("  - robustness_results")
    print("  - grid_results")
    print("\n" + "="*70)
    print("LOAD_SAVED_RESULTS = True")
    print("Training cells will be SKIPPED automatically.")
    print("You can safely 'Run All' - analysis cells will use saved results.")
    print("="*70)
else:
    print("LOAD_SAVED_RESULTS = False")
    print("Training cells will run and generate fresh results.")
    print("This will take significant time.")

LOAD_SAVED_RESULTS = False
Training cells will run and generate fresh results.
This will take significant time.


## Cell 7: Alpha-Sampling Interaction Analysis

**Important Issue**: The alpha parameter and sampling ratio interact in non-obvious ways.

With 1:4 sampling and alpha=0.5:
- Each batch has 1 positive (weight: 0.5) and 4 negatives (weight: 0.5 each)
- **Effective ratio**: (0.5 × 4) / 0.5 = 4:1 (negatives still get 4x more weight!)

For balanced weighting with 1:N sampling:
- Need alpha = N / (N + 1)
- Example: 1:4 -> alpha = 0.8, 1:10 -> alpha = 0.909, 1:50 -> alpha = 0.98

This explains why alpha=0.25 (from computer vision) may not be optimal for NCF.

In [8]:
print("="*70)
print("ALPHA-SAMPLING INTERACTION ANALYSIS")
print("="*70)

# Analyze interaction for various alpha values
interaction_df = analyze_alpha_sampling_interaction(
    neg_ratios=SAMPLING_RATIOS,
    alphas=[0.25, 0.5, 0.75]
)
print("\n" + interaction_df.to_string(index=False))

print("\n" + "-"*70)
print("BALANCED ALPHA VALUES (for effective ratio = 1:1)")
print("-"*70)

for ratio in SAMPLING_RATIOS:
    balanced = get_balanced_alpha(ratio)
    actual_ratio = compute_effective_class_ratio(ALPHA, ratio)
    print(f"1:{ratio} sampling -> balanced alpha = {balanced:.3f}")
    print(f"  Using alpha={ALPHA} -> effective ratio = {actual_ratio:.1f}:1")
    print()

ALPHA-SAMPLING INTERACTION ANALYSIS

neg_ratio  alpha effective_ratio balanced_alpha  is_balanced
      1:4   0.25          12.0:1           0.80        False
      1:4   0.50           4.0:1           0.80        False
      1:4   0.75           1.3:1           0.80        False
     1:10   0.25          30.0:1           0.91        False
     1:10   0.50          10.0:1           0.91        False
     1:10   0.75           3.3:1           0.91        False
     1:50   0.25         150.0:1           0.98        False
     1:50   0.50          50.0:1           0.98        False
     1:50   0.75          16.7:1           0.98        False

----------------------------------------------------------------------
BALANCED ALPHA VALUES (for effective ratio = 1:1)
----------------------------------------------------------------------
1:4 sampling -> balanced alpha = 0.800
  Using alpha=0.25 -> effective ratio = 12.0:1

1:10 sampling -> balanced alpha = 0.909
  Using alpha=0.25 -> effective r

## Experiment 1: NeuMF-BCE Baseline (1:10 Sampling)

Train standard NeuMF with Binary Cross-Entropy loss.

This serves as the baseline for H1 (efficacy hypothesis).

In [9]:
# Skip if using saved results
if LOAD_SAVED_RESULTS:
    print("="*70)
    print(f"EXPERIMENT 1: NeuMF-BCE Baseline (1:{PRIMARY_RATIO} sampling)")
    print("="*70)
    print("SKIPPED - Using saved results")
    print(f"  Test NDCG@10: {result_bce['test_result'].get('ndcg@10', 0):.4f}")
    print(f"  Test HR@10: {result_bce['test_result'].get('hit@10', 0):.4f}")
else:
    print("="*70)
    print(f"EXPERIMENT 1: NeuMF-BCE Baseline (1:{PRIMARY_RATIO} sampling)")
    print("="*70)

    result_bce = run_recbole(
        model='NeuMF',
        dataset=DATASET,
        config_dict=configs[PRIMARY_RATIO]
    )

    print("\nNeuMF-BCE Results:")
    print(f"  Best Validation NDCG@10: {result_bce['best_valid_score']:.4f}")
    print(f"  Test HR@10: {result_bce['test_result'].get('hit@10', 0):.4f}")
    print(f"  Test NDCG@10: {result_bce['test_result'].get('ndcg@10', 0):.4f}")

EXPERIMENT 1: NeuMF-BCE Baseline (1:10 sampling)


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c


NeuMF-BCE Results:
  Best Validation NDCG@10: 0.0675
  Test HR@10: 0.1125
  Test NDCG@10: 0.0579


## Experiment 2: NeuMF with Alpha-Balanced BCE (1:10 Sampling)

**NEW CONTROL EXPERIMENT**: Train NeuMF with alpha-balanced BCE (Focal Loss with gamma=0).

This isolates the **focusing effect** (gamma > 0) from simple **class weighting** (alpha).

- If Alpha-BCE ≈ FL: Improvement comes mainly from class weighting (H3 false)
- If Alpha-BCE < FL: Focusing effect is necessary (H3 true)

We use alpha=0.25 (same as Focal Loss) for direct comparison.

In [10]:
# Skip if using saved results
if LOAD_SAVED_RESULTS:
    print("="*70)
    print(f"EXPERIMENT 2: NeuMF-AlphaBCE Control (1:{PRIMARY_RATIO} sampling)")
    print("="*70)
    print("SKIPPED - Using saved results")
    print(f"  Test NDCG@10: {result_alpha_bce['test_result'].get('ndcg@10', 0):.4f}")
    print(f"  Test HR@10: {result_alpha_bce['test_result'].get('hit@10', 0):.4f}")
else:
    print("="*70)
    print(f"EXPERIMENT 2: NeuMF-AlphaBCE Control (1:{PRIMARY_RATIO} sampling)")
    print("="*70)
    print(f"Alpha-Balanced BCE: gamma=0, alpha={ALPHA}")
    print("This isolates class weighting from the focusing effect.\n")

    result_alpha_bce = train_neumf_alpha_bce(
        config_dict=configs[PRIMARY_RATIO],
        dataset=DATASET,
        alpha=ALPHA,
        seed=42,
        track_dynamics=False
    )

    print("\nNeuMF-AlphaBCE Results:")
    print(f"  Best Validation NDCG@10: {result_alpha_bce['best_valid_score']:.4f}")
    print(f"  Test HR@10: {result_alpha_bce['test_result'].get('hit@10', 0):.4f}")
    print(f"  Test NDCG@10: {result_alpha_bce['test_result'].get('ndcg@10', 0):.4f}")

EXPERIMENT 2: NeuMF-AlphaBCE Control (1:10 sampling)
Alpha-Balanced BCE: gamma=0, alpha=0.25
This isolates class weighting from the focusing effect.



/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

Initialized NeuMF with Alpha-Balanced BCE (alpha=0.25)


/usr/local/lib/python3.12/dist-packages/recbole/trainer/trainer.py:235: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler(enabled=self.enable_scaler)



NeuMF-AlphaBCE Results:
  Best Validation NDCG@10: 0.0682
  Test HR@10: 0.1231
  Test NDCG@10: 0.0598


## Experiment 3: NeuMF with Focal Loss (1:10 Sampling)

Train NeuMF with Focal Loss (gamma=2.0, alpha=0.25).

This is the main proposed method for H1.

In [11]:
# Skip if using saved results
if LOAD_SAVED_RESULTS:
    print("="*70)
    print(f"EXPERIMENT 3: NeuMF-FocalLoss (1:{PRIMARY_RATIO} sampling)")
    print("="*70)
    print("SKIPPED - Using saved results")
    print(f"  Test NDCG@10: {result_focal['test_result'].get('ndcg@10', 0):.4f}")
    print(f"  Test HR@10: {result_focal['test_result'].get('hit@10', 0):.4f}")
else:
    print("="*70)
    print(f"EXPERIMENT 3: NeuMF-FocalLoss (1:{PRIMARY_RATIO} sampling)")
    print("="*70)
    print(f"Focal Loss: gamma={GAMMA}, alpha={ALPHA}\n")

    result_focal = train_neumf_focal_loss(
        config_dict=configs[PRIMARY_RATIO],
        dataset=DATASET,
        gamma=GAMMA,
        alpha=ALPHA,
        seed=42,
        track_dynamics=False
    )

    print("\nNeuMF-FocalLoss Results:")
    print(f"  Best Validation NDCG@10: {result_focal['best_valid_score']:.4f}")
    print(f"  Test HR@10: {result_focal['test_result'].get('hit@10', 0):.4f}")
    print(f"  Test NDCG@10: {result_focal['test_result'].get('ndcg@10', 0):.4f}")

EXPERIMENT 3: NeuMF-FocalLoss (1:10 sampling)
Focal Loss: gamma=2.0, alpha=0.25



/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

Initialized NeuMF with Focal Loss (gamma=2.0, alpha=0.25)


/usr/local/lib/python3.12/dist-packages/recbole/trainer/trainer.py:235: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler(enabled=self.enable_scaler)



NeuMF-FocalLoss Results:
  Best Validation NDCG@10: 0.0637
  Test HR@10: 0.1115
  Test NDCG@10: 0.0564


## Hypothesis Testing: Primary Results (1:10 Sampling)

Compare BCE vs Alpha-BCE vs Focal Loss to test H1 and H3.

In [12]:
print("="*70)
print(f"HYPOTHESIS TESTING: Primary Results (1:{PRIMARY_RATIO} sampling)")
print("="*70)

# Create comparison table
comparison_df = create_comparison_table(
    results_list=[result_bce, result_alpha_bce, result_focal],
    model_names=['BCE', 'AlphaBCE', 'FocalLoss']
)
print("\n" + comparison_df.to_string(index=False))

# Compute improvements
print("\n" + "-"*70)
print("IMPROVEMENT ANALYSIS")
print("-"*70)

# H1: Focal Loss vs BCE
print("\nH1: Focal Loss vs BCE (Efficacy Hypothesis)")
improvements_fl_vs_bce = compute_improvement(result_bce, result_focal)
for metric in ['ndcg@10', 'hit@10']:
    imp = improvements_fl_vs_bce[metric]
    print(f"  {metric.upper()}: {imp['baseline']:.4f} -> {imp['comparison']:.4f} ({imp['pct_change']:+.2f}%)")

# H3: Focal Loss vs Alpha-BCE (Mechanism Hypothesis)
print("\nH3: Focal Loss vs Alpha-BCE (Mechanism - Focusing Effect)")
improvements_fl_vs_alpha = compute_improvement(result_alpha_bce, result_focal)
for metric in ['ndcg@10', 'hit@10']:
    imp = improvements_fl_vs_alpha[metric]
    print(f"  {metric.upper()}: {imp['baseline']:.4f} -> {imp['comparison']:.4f} ({imp['pct_change']:+.2f}%)")

# Interpretation
print("\n" + "-"*70)
print("INTERPRETATION")
print("-"*70)

ndcg_fl = result_focal['test_result'].get('ndcg@10', 0)
ndcg_bce = result_bce['test_result'].get('ndcg@10', 0)
ndcg_alpha = result_alpha_bce['test_result'].get('ndcg@10', 0)

if ndcg_fl > ndcg_bce:
    print("✓ H1 SUPPORTED: Focal Loss improves over BCE")
else:
    print("✗ H1 NOT SUPPORTED: Focal Loss does not improve over BCE")

if ndcg_fl > ndcg_alpha:
    print("✓ H3 SUPPORTED: Focusing effect (gamma > 0) is necessary beyond class weighting")
else:
    print("✗ H3 NOT SUPPORTED: Class weighting alone (Alpha-BCE) is sufficient")

if ndcg_alpha > ndcg_bce:
    print("  Note: Alpha-BCE also improves over BCE, suggesting class weighting helps")
else:
    print("  Note: Alpha-BCE does not improve over BCE")

HYPOTHESIS TESTING: Primary Results (1:10 sampling)

 Metric    BCE AlphaBCE FocalLoss
  HIT@5 0.0616   0.0669    0.0701
 HIT@10 0.1125   0.1231    0.1115
 HIT@20 0.1794   0.1921    0.1921
 NDCG@5 0.0415   0.0416    0.0433
NDCG@10 0.0579   0.0598    0.0564
NDCG@20 0.0746   0.0770    0.0767

----------------------------------------------------------------------
IMPROVEMENT ANALYSIS
----------------------------------------------------------------------

H1: Focal Loss vs BCE (Efficacy Hypothesis)
  NDCG@10: 0.0579 -> 0.0564 (-2.59%)
  HIT@10: 0.1125 -> 0.1115 (-0.89%)

H3: Focal Loss vs Alpha-BCE (Mechanism - Focusing Effect)
  NDCG@10: 0.0598 -> 0.0564 (-5.69%)
  HIT@10: 0.1231 -> 0.1115 (-9.42%)

----------------------------------------------------------------------
INTERPRETATION
----------------------------------------------------------------------
✗ H1 NOT SUPPORTED: Focal Loss does not improve over BCE
✗ H3 NOT SUPPORTED: Class weighting alone (Alpha-BCE) is sufficient
  Note: Alph

## Robustness Study: Multiple Sampling Ratios (H2)

Test whether Focal Loss improvements are robust across different sampling ratios.

We compare BCE vs Focal Loss at 1:4, 1:10, and 1:50 sampling.

In [13]:
# Skip if using saved results
if LOAD_SAVED_RESULTS:
    print("="*70)
    print("H2: ROBUSTNESS STUDY - Multiple Sampling Ratios")
    print("="*70)
    print("SKIPPED - Using saved results\n")

    # Show saved results summary
    for ratio in SAMPLING_RATIOS:
        bce_ndcg = robustness_results['bce'][ratio]['test_result'].get('ndcg@10', 0)
        fl_ndcg = robustness_results['focal'][ratio]['test_result'].get('ndcg@10', 0)
        imp = (fl_ndcg - bce_ndcg) / bce_ndcg * 100 if bce_ndcg > 0 else 0
        print(f"1:{ratio}: BCE={bce_ndcg:.4f}, FL={fl_ndcg:.4f} ({imp:+.1f}%)")
else:
    print("="*70)
    print("H2: ROBUSTNESS STUDY - Multiple Sampling Ratios")
    print("="*70)
    print("Testing BCE vs Focal Loss at 1:4, 1:10, and 1:50 sampling\n")

    robustness_results = {
        'bce': {},
        'focal': {}
    }

    for ratio in SAMPLING_RATIOS:
        print(f"\n{'='*70}")
        print(f"Sampling Ratio: 1:{ratio}")
        print('='*70)

        # Skip if we already trained at PRIMARY_RATIO
        if ratio == PRIMARY_RATIO:
            print(f"Using existing results from primary experiment")
            robustness_results['bce'][ratio] = result_bce
            robustness_results['focal'][ratio] = result_focal
            continue

        # Train BCE
        print(f"\nTraining NeuMF-BCE (1:{ratio})...")
        result_bce_ratio = run_recbole(
            model='NeuMF',
            dataset=DATASET,
            config_dict=configs[ratio]
        )
        robustness_results['bce'][ratio] = result_bce_ratio

        # Train Focal Loss
        print(f"Training NeuMF-FocalLoss (1:{ratio})...")
        result_focal_ratio = train_neumf_focal_loss(
            config_dict=configs[ratio],
            dataset=DATASET,
            gamma=GAMMA,
            alpha=ALPHA,
            seed=42,
            track_dynamics=False
        )
        robustness_results['focal'][ratio] = result_focal_ratio

        # Show results
        bce_ndcg = result_bce_ratio['test_result'].get('ndcg@10', 0)
        fl_ndcg = result_focal_ratio['test_result'].get('ndcg@10', 0)
        improvement = (fl_ndcg - bce_ndcg) / bce_ndcg * 100 if bce_ndcg > 0 else 0

        print(f"\nResults for 1:{ratio} sampling:")
        print(f"  BCE NDCG@10: {bce_ndcg:.4f}")
        print(f"  Focal NDCG@10: {fl_ndcg:.4f}")
        print(f"  Improvement: {improvement:+.2f}%")

H2: ROBUSTNESS STUDY - Multiple Sampling Ratios
Testing BCE vs Focal Loss at 1:4, 1:10, and 1:50 sampling


Sampling Ratio: 1:4

Training NeuMF-BCE (1:4)...


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

Training NeuMF-FocalLoss (1:4)...


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

Initialized NeuMF with Focal Loss (gamma=2.0, alpha=0.25)


...

Training NeuMF-FocalLoss (1:50)...


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

Initialized NeuMF with Focal Loss (gamma=2.0, alpha=0.25)


/usr/local/lib/python3.12/dist-packages/recbole/trainer/trainer.py:235: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler(enabled=self.enable_scaler)



Results for 1:50 sampling:
  BCE NDCG@10: 0.0530
  Focal NDCG@10: 0.0672
  Improvement: +26.79%


## Robustness Study: Summary Table

In [14]:
print("="*70)
print("ROBUSTNESS STUDY SUMMARY")
print("="*70)

# Create summary table
summary_data = []
for ratio in SAMPLING_RATIOS:
    bce_result = robustness_results['bce'][ratio]
    fl_result = robustness_results['focal'][ratio]

    bce_ndcg = bce_result['test_result'].get('ndcg@10', 0)
    fl_ndcg = fl_result['test_result'].get('ndcg@10', 0)
    bce_hr = bce_result['test_result'].get('hit@10', 0)
    fl_hr = fl_result['test_result'].get('hit@10', 0)

    ndcg_imp = (fl_ndcg - bce_ndcg) / bce_ndcg * 100 if bce_ndcg > 0 else 0
    hr_imp = (fl_hr - bce_hr) / bce_hr * 100 if bce_hr > 0 else 0

    summary_data.append({
        'Sampling': f'1:{ratio}',
        'BCE_NDCG@10': f'{bce_ndcg:.4f}',
        'FL_NDCG@10': f'{fl_ndcg:.4f}',
        'NDCG_Improvement': f'{ndcg_imp:+.2f}%',
        'BCE_HR@10': f'{bce_hr:.4f}',
        'FL_HR@10': f'{fl_hr:.4f}',
        'HR_Improvement': f'{hr_imp:+.2f}%'
    })

summary_df = pd.DataFrame(summary_data)
print("\n" + summary_df.to_string(index=False))

# H2 interpretation
print("\n" + "-"*70)
print("H2 INTERPRETATION (Robustness)")
print("-"*70)

improvements = []
for ratio in SAMPLING_RATIOS:
    bce_ndcg = robustness_results['bce'][ratio]['test_result'].get('ndcg@10', 0)
    fl_ndcg = robustness_results['focal'][ratio]['test_result'].get('ndcg@10', 0)
    improvements.append(fl_ndcg > bce_ndcg)

if all(improvements):
    print("✓ H2 STRONGLY SUPPORTED: Focal Loss improves over BCE at ALL sampling ratios")
elif sum(improvements) >= 2:
    print("✓ H2 PARTIALLY SUPPORTED: Focal Loss improves at most sampling ratios")
else:
    print("✗ H2 NOT SUPPORTED: Focal Loss improvements are not robust")

print(f"\nFocal Loss wins at {sum(improvements)}/{len(SAMPLING_RATIOS)} sampling ratios")

ROBUSTNESS STUDY SUMMARY

Sampling BCE_NDCG@10 FL_NDCG@10 NDCG_Improvement BCE_HR@10 FL_HR@10 HR_Improvement
     1:4      0.0640     0.0604           -5.62%    0.1263   0.1242         -1.66%
    1:10      0.0579     0.0564           -2.59%    0.1125   0.1115         -0.89%
    1:50      0.0530     0.0672          +26.79%    0.1083   0.1231        +13.67%

----------------------------------------------------------------------
H2 INTERPRETATION (Robustness)
----------------------------------------------------------------------
✗ H2 NOT SUPPORTED: Focal Loss improvements are not robust

Focal Loss wins at 1/3 sampling ratios


## Training Dynamics Analysis

**NEW ANALYSIS**: Track training dynamics to validate the Focal Loss mechanism.

We track loss contribution by confidence bin to verify:
1. Focal Loss down-weights easy examples (high confidence)
2. Focal Loss emphasizes hard examples (low confidence)
3. This leads to better focusing on difficult instances

This provides direct evidence for the claimed mechanism.

In [15]:
# Skip if using saved results (dynamics data not saved)
if LOAD_SAVED_RESULTS:
    print("="*70)
    print("TRAINING DYNAMICS ANALYSIS")
    print("="*70)
    print("SKIPPED - Dynamics tracking requires fresh training")
    print("Set LOAD_SAVED_RESULTS = False to run dynamics analysis")
    bce_dynamics = {'dynamics': None}
    focal_dynamics = {'dynamics': None}
else:
    print("="*70)
    print("TRAINING DYNAMICS ANALYSIS")
    print("="*70)
    print("Re-training models with dynamics tracking enabled...\n")

    # Train BCE with dynamics tracking
    print("Training BCE with dynamics tracking...")
    # Note: Standard BCE from RecBole doesn't support dynamics tracking
    # We approximate by using Alpha-BCE with alpha=0.5
    bce_dynamics = train_neumf_alpha_bce(
        config_dict=configs[PRIMARY_RATIO],
        dataset=DATASET,
        alpha=0.5,  # Standard BCE
        seed=42,
        track_dynamics=True
    )

    # Train Focal Loss with dynamics tracking
    print("\nTraining Focal Loss with dynamics tracking...")
    focal_dynamics = train_neumf_focal_loss(
        config_dict=configs[PRIMARY_RATIO],
        dataset=DATASET,
        gamma=GAMMA,
        alpha=ALPHA,
        seed=42,
        track_dynamics=True
    )

    print("\nTraining complete. Analyzing dynamics...")

TRAINING DYNAMICS ANALYSIS
Re-training models with dynamics tracking enabled...

Training BCE with dynamics tracking...


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

Initialized NeuMF with Alpha-Balanced BCE (alpha=0.5)


/usr/local/lib/python3.12/dist-packages/recbole/trainer/trainer.py:235: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler(enabled=self.enable_scaler)



Training Focal Loss with dynamics tracking...


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

Initialized NeuMF with Focal Loss (gamma=2.0, alpha=0.25)


/usr/local/lib/python3.12/dist-packages/recbole/trainer/trainer.py:235: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler(enabled=self.enable_scaler)



Training complete. Analyzing dynamics...


## Training Dynamics: Results

In [16]:
print("="*70)
print("TRAINING DYNAMICS: Loss by Confidence Bin")
print("="*70)

if 'dynamics' in bce_dynamics and 'dynamics' in focal_dynamics:
    bce_dyn_df = bce_dynamics['dynamics']
    focal_dyn_df = focal_dynamics['dynamics']

    # Show final epoch dynamics
    if len(bce_dyn_df) > 0 and len(focal_dyn_df) > 0:
        print("\nBCE - Final Epoch Loss by Confidence Bin:")
        print(bce_dyn_df.tail(1).to_string(index=False))

        print("\nFocal Loss - Final Epoch Loss by Confidence Bin:")
        print(focal_dyn_df.tail(1).to_string(index=False))

        print("\n" + "-"*70)
        print("INTERPRETATION")
        print("-"*70)
        print("Expected pattern for Focal Loss:")
        print("  - LOW loss in high confidence bins [0.8,1.0) - easy examples down-weighted")
        print("  - HIGH loss in low confidence bins [0.0,0.4) - hard examples emphasized")
        print("\nThis validates the focusing mechanism.")
    else:
        print("Dynamics tracking did not record data. Check model implementation.")
else:
    print("Dynamics tracking not available. Models may not support this feature.")
    print("This is optional analysis - main results are still valid.")

TRAINING DYNAMICS: Loss by Confidence Bin
Dynamics tracking did not record data. Check model implementation.


## Results Summary: Full Comparison

Complete results across all experiments.

In [17]:
print("="*70)
print("FINAL RESULTS SUMMARY")
print("="*70)

print(f"\nDataset: {DATASET}")
print(f"Primary Experiment: 1:{PRIMARY_RATIO} sampling")
print(f"Focal Loss: gamma={GAMMA}, alpha={ALPHA}")

print("\n" + "-"*70)
print(f"PRIMARY RESULTS (1:{PRIMARY_RATIO} sampling)")
print("-"*70)

primary_comparison = create_comparison_table(
    results_list=[result_bce, result_alpha_bce, result_focal],
    model_names=['BCE', 'AlphaBCE', 'FocalLoss']
)
print("\n" + primary_comparison.to_string(index=False))

print("\n" + "-"*70)
print("ROBUSTNESS ACROSS SAMPLING RATIOS")
print("-"*70)
print("\n" + summary_df.to_string(index=False))

print("\n" + "="*70)
print("HYPOTHESIS TESTING CONCLUSIONS")
print("="*70)

# H1
ndcg_fl = result_focal['test_result'].get('ndcg@10', 0)
ndcg_bce = result_bce['test_result'].get('ndcg@10', 0)
h1_imp = (ndcg_fl - ndcg_bce) / ndcg_bce * 100 if ndcg_bce > 0 else 0

print("\nH1 (Efficacy): Focal Loss improves NeuMF over BCE")
if ndcg_fl > ndcg_bce:
    print(f"  ✓ SUPPORTED: {h1_imp:+.2f}% improvement in NDCG@10")
else:
    print(f"  ✗ NOT SUPPORTED: {h1_imp:+.2f}% change in NDCG@10")

# H2
robust_count = sum([robustness_results['focal'][r]['test_result'].get('ndcg@10', 0) >
                    robustness_results['bce'][r]['test_result'].get('ndcg@10', 0)
                    for r in SAMPLING_RATIOS])

print(f"\nH2 (Robustness): Improvements are robust across sampling ratios")
if robust_count == len(SAMPLING_RATIOS):
    print(f"  ✓ STRONGLY SUPPORTED: FL wins at {robust_count}/{len(SAMPLING_RATIOS)} ratios")
elif robust_count >= 2:
    print(f"  ✓ PARTIALLY SUPPORTED: FL wins at {robust_count}/{len(SAMPLING_RATIOS)} ratios")
else:
    print(f"  ✗ NOT SUPPORTED: FL wins at only {robust_count}/{len(SAMPLING_RATIOS)} ratios")

# H3
ndcg_alpha = result_alpha_bce['test_result'].get('ndcg@10', 0)
h3_imp = (ndcg_fl - ndcg_alpha) / ndcg_alpha * 100 if ndcg_alpha > 0 else 0

print(f"\nH3 (Mechanism): Focusing effect (gamma > 0) is necessary beyond class weighting")
if ndcg_fl > ndcg_alpha:
    print(f"  ✓ SUPPORTED: FL gains {h3_imp:+.2f}% over Alpha-BCE")
else:
    print(f"  ✗ NOT SUPPORTED: Alpha-BCE is sufficient ({h3_imp:+.2f}%)")

print("\n" + "="*70)

FINAL RESULTS SUMMARY

Dataset: ml-100k
Primary Experiment: 1:10 sampling
Focal Loss: gamma=2.0, alpha=0.25

----------------------------------------------------------------------
PRIMARY RESULTS (1:10 sampling)
----------------------------------------------------------------------

 Metric    BCE AlphaBCE FocalLoss
  HIT@5 0.0616   0.0669    0.0701
 HIT@10 0.1125   0.1231    0.1115
 HIT@20 0.1794   0.1921    0.1921
 NDCG@5 0.0415   0.0416    0.0433
NDCG@10 0.0579   0.0598    0.0564
NDCG@20 0.0746   0.0770    0.0767

----------------------------------------------------------------------
ROBUSTNESS ACROSS SAMPLING RATIOS
----------------------------------------------------------------------

Sampling BCE_NDCG@10 FL_NDCG@10 NDCG_Improvement BCE_HR@10 FL_HR@10 HR_Improvement
     1:4      0.0640     0.0604           -5.62%    0.1263   0.1242         -1.66%
    1:10      0.0579     0.0564           -2.59%    0.1125   0.1115         -0.89%
    1:50      0.0530     0.0672          +26.79%   

## Optional: Grid Search for Hyperparameter Tuning

**IMPORTANT**: This is computationally expensive. Only run if you want to optimize gamma and alpha.

The grid search tests:
- Multiple gamma values: [0.5, 1.0, 2.0, 3.0]
- Multiple alpha values: [0.25, 0.5, 0.75]
- Multiple sampling ratios: [4, 10, 50]

**Uncomment to run**

In [ ]:
# ============================================
# GRID SEARCH (OPTIONAL - COMPUTATIONALLY EXPENSIVE)
# ============================================

GAMMA_VALUES = [0.5, 1.0, 2.0, 3.0]
ALPHA_VALUES = [0.25, 0.5, 0.75]

# Skip if using saved results
if LOAD_SAVED_RESULTS:
    print("="*70)
    print("GRID SEARCH: Hyperparameter Tuning")
    print("="*70)
    print("SKIPPED - Using saved results")
    print(f"Loaded {len(grid_results)} pre-computed configurations\n")

    # Convert to DataFrame for display
    grid_df = pd.DataFrame(grid_results)

    # Show best per ratio
    print("Best configurations per sampling ratio:")
    for ratio in SAMPLING_RATIOS:
        ratio_results = [r for r in grid_results if r['ratio'] == ratio]
        best = max(ratio_results, key=lambda x: x['ndcg@10'])
        print(f"  1:{ratio}: gamma={best['gamma']}, alpha={best['alpha']} -> NDCG@10={best['ndcg@10']:.4f}")
else:
    print("="*70)
    print("GRID SEARCH: Hyperparameter Tuning")
    print("="*70)
    print(f"Testing {len(GAMMA_VALUES)} gamma x {len(ALPHA_VALUES)} alpha x {len(SAMPLING_RATIOS)} ratios")
    print(f"Total: {len(GAMMA_VALUES) * len(ALPHA_VALUES) * len(SAMPLING_RATIOS)} experiments\n")

    grid_results = []

    for ratio in SAMPLING_RATIOS:
        print(f"\nSampling Ratio: 1:{ratio}")
        print("-"*70)

        for gamma in GAMMA_VALUES:
            for alpha in ALPHA_VALUES:
                print(f"Training: gamma={gamma}, alpha={alpha}")

                result = train_neumf_focal_loss(
                    config_dict=configs[ratio],
                    dataset=DATASET,
                    gamma=gamma,
                    alpha=alpha,
                    seed=42,
                    track_dynamics=False
                )

                ndcg10 = result['test_result'].get('ndcg@10', 0)
                hr10 = result['test_result'].get('hit@10', 0)

                grid_results.append({
                    'ratio': ratio,
                    'gamma': gamma,
                    'alpha': alpha,
                    'ndcg@10': ndcg10,
                    'hit@10': hr10
                })

                print(f"  NDCG@10: {ndcg10:.4f}, HR@10: {hr10:.4f}")

    # Show grid search results
    grid_df = pd.DataFrame(grid_results)

    print("\n" + "="*70)
    print("GRID SEARCH RESULTS")
    print("="*70)
    print("\n" + grid_df.to_string(index=False))

    # Find best configuration for each ratio
    print("\n" + "-"*70)
    print("BEST CONFIGURATIONS PER SAMPLING RATIO")
    print("-"*70)

    for ratio in SAMPLING_RATIOS:
        ratio_results = grid_df[grid_df['ratio'] == ratio]
        best_idx = ratio_results['ndcg@10'].idxmax()
        best_row = ratio_results.loc[best_idx]

        print(f"\n1:{ratio} sampling:")
        print(f"  Best: gamma={best_row['gamma']}, alpha={best_row['alpha']}")
        print(f"  NDCG@10: {best_row['ndcg@10']:.4f}")
        print(f"  HR@10: {best_row['hit@10']:.4f}")

GRID SEARCH: Hyperparameter Tuning
Testing 4 gamma x 3 alpha x 3 ratios
Total: 36 experiments


Sampling Ratio: 1:4
----------------------------------------------------------------------
Training: gamma=0.5, alpha=0.25


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

Initialized NeuMF with Focal Loss (gamma=0.5, alpha=0.25)


/usr/local/lib/python3.12/dist-packages/recbole/trainer/trainer.py:235: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler(enabled=self.enable_scaler)


  NDCG@10: 0.0636, HR@10: 0.1253
Training: gamma=0.5, alpha=0.5


...

/usr/local/lib/python3.12/dist-packages/recbole/trainer/trainer.py:235: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler(enabled=self.enable_scaler)


  NDCG@10: 0.0708, HR@10: 0.1316
Training: gamma=0.5, alpha=0.75


/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.12/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

Initialized NeuMF with Focal Loss (gamma=0.5, alpha=0.75)


/usr/local/lib/python3.12/dist-packages/recbole/trainer/trainer.py:235: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler(enabled=self.enable_scaler)


## Statistical Analysis: Existing Results

Before running multi-seed experiments, let's analyze the statistical patterns in our existing results from the grid search. While single-seed results don't provide true statistical significance, we can:

1. **Aggregate across hyperparameter configurations** - Treat different (gamma, alpha) combinations as repeated measurements
2. **Sign test across sampling ratios** - Check consistency of FL > BCE across different conditions
3. **Effect size estimation** - Quantify the magnitude of improvements

This provides preliminary evidence before investing in full multi-seed experiments.

In [ ]:
# ============================================
# STATISTICAL ANALYSIS OF EXISTING RESULTS
# Using Grid Search and Robustness Study Data
# ============================================
from scipy import stats
import numpy as np

print("="*70)
print("STATISTICAL ANALYSIS: Existing Experimental Results")
print("="*70)

# ============================================
# 1. AGGREGATE GRID SEARCH RESULTS
# ============================================
print("\n" + "-"*70)
print("1. GRID SEARCH ANALYSIS")
print("-"*70)

# Get BCE baselines for each sampling ratio (from robustness study)
bce_baselines = {}
for ratio in SAMPLING_RATIOS:
    bce_baselines[ratio] = {
        'ndcg@10': robustness_results['bce'][ratio]['test_result'].get('ndcg@10', 0),
        'hit@10': robustness_results['bce'][ratio]['test_result'].get('hit@10', 0)
    }
    print(f"BCE Baseline (1:{ratio}): NDCG@10={bce_baselines[ratio]['ndcg@10']:.4f}, HR@10={bce_baselines[ratio]['hit@10']:.4f}")

# Analyze grid search results
print("\nFocal Loss configurations that beat BCE baseline:")
fl_wins = {'ndcg@10': 0, 'hit@10': 0}
fl_total = 0
improvements_ndcg = []
improvements_hr = []

for row in grid_results:
    ratio = row['ratio']
    fl_ndcg = row['ndcg@10']
    fl_hr = row['hit@10']
    bce_ndcg = bce_baselines[ratio]['ndcg@10']
    bce_hr = bce_baselines[ratio]['hit@10']

    fl_total += 1
    if fl_ndcg > bce_ndcg:
        fl_wins['ndcg@10'] += 1
    if fl_hr > bce_hr:
        fl_wins['hit@10'] += 1

    # Store improvements for effect size calculation
    improvements_ndcg.append((fl_ndcg - bce_ndcg) / bce_ndcg * 100)
    improvements_hr.append((fl_hr - bce_hr) / bce_hr * 100)

print(f"\nFocal Loss wins (NDCG@10): {fl_wins['ndcg@10']}/{fl_total} ({fl_wins['ndcg@10']/fl_total*100:.1f}%)")
print(f"Focal Loss wins (HR@10):   {fl_wins['hit@10']}/{fl_total} ({fl_wins['hit@10']/fl_total*100:.1f}%)")

# ============================================
# 2. SIGN TEST (Non-parametric)
# ============================================
print("\n" + "-"*70)
print("2. SIGN TEST (Binomial Test)")
print("-"*70)
print("H0: P(FL > BCE) = 0.5 (Focal Loss is no better than random)")
print("H1: P(FL > BCE) > 0.5 (Focal Loss systematically improves over BCE)")

# Sign test for NDCG@10
n_positive_ndcg = fl_wins['ndcg@10']
n_total = fl_total
p_value_ndcg = stats.binom_test(n_positive_ndcg, n_total, 0.5, alternative='greater')
print(f"\nNDCG@10: {n_positive_ndcg}/{n_total} positive differences")
print(f"  Binomial test p-value: {p_value_ndcg:.4f}")
print(f"  Significant (p < 0.05): {'YES' if p_value_ndcg < 0.05 else 'NO'}")

# Sign test for HR@10
n_positive_hr = fl_wins['hit@10']
p_value_hr = stats.binom_test(n_positive_hr, n_total, 0.5, alternative='greater')
print(f"\nHR@10: {n_positive_hr}/{n_total} positive differences")
print(f"  Binomial test p-value: {p_value_hr:.4f}")
print(f"  Significant (p < 0.05): {'YES' if p_value_hr < 0.05 else 'NO'}")

# ============================================
# 3. EFFECT SIZE ANALYSIS
# ============================================
print("\n" + "-"*70)
print("3. EFFECT SIZE ANALYSIS")
print("-"*70)

improvements_ndcg = np.array(improvements_ndcg)
improvements_hr = np.array(improvements_hr)

print(f"\nNDCG@10 Improvement over BCE:")
print(f"  Mean:   {np.mean(improvements_ndcg):+.2f}%")
print(f"  Median: {np.median(improvements_ndcg):+.2f}%")
print(f"  Std:    {np.std(improvements_ndcg):.2f}%")
print(f"  Range:  [{np.min(improvements_ndcg):+.2f}%, {np.max(improvements_ndcg):+.2f}%]")

print(f"\nHR@10 Improvement over BCE:")
print(f"  Mean:   {np.mean(improvements_hr):+.2f}%")
print(f"  Median: {np.median(improvements_hr):+.2f}%")
print(f"  Std:    {np.std(improvements_hr):.2f}%")
print(f"  Range:  [{np.min(improvements_hr):+.2f}%, {np.max(improvements_hr):+.2f}%]")

# Cohen's d effect size (treating grid configs as samples)
cohens_d_ndcg = np.mean(improvements_ndcg) / np.std(improvements_ndcg) if np.std(improvements_ndcg) > 0 else 0
cohens_d_hr = np.mean(improvements_hr) / np.std(improvements_hr) if np.std(improvements_hr) > 0 else 0

def interpret_cohens_d(d):
    d = abs(d)
    if d < 0.2: return "negligible"
    elif d < 0.5: return "small"
    elif d < 0.8: return "medium"
    else: return "large"

print(f"\nCohen's d effect size:")
print(f"  NDCG@10: {cohens_d_ndcg:.3f} ({interpret_cohens_d(cohens_d_ndcg)})")
print(f"  HR@10:   {cohens_d_hr:.3f} ({interpret_cohens_d(cohens_d_hr)})")

# ============================================
# 4. ONE-SAMPLE T-TEST (Improvements > 0)
# ============================================
print("\n" + "-"*70)
print("4. ONE-SAMPLE T-TEST (Mean Improvement > 0)")
print("-"*70)
print("H0: Mean improvement = 0")
print("H1: Mean improvement > 0")

t_stat_ndcg, p_value_t_ndcg = stats.ttest_1samp(improvements_ndcg, 0)
p_value_t_ndcg_one = p_value_t_ndcg / 2 if t_stat_ndcg > 0 else 1 - p_value_t_ndcg / 2

print(f"\nNDCG@10:")
print(f"  t-statistic: {t_stat_ndcg:.3f}")
print(f"  p-value (one-tailed): {p_value_t_ndcg_one:.4f}")
print(f"  Significant (p < 0.05): {'YES' if p_value_t_ndcg_one < 0.05 else 'NO'}")

t_stat_hr, p_value_t_hr = stats.ttest_1samp(improvements_hr, 0)
p_value_t_hr_one = p_value_t_hr / 2 if t_stat_hr > 0 else 1 - p_value_t_hr / 2

print(f"\nHR@10:")
print(f"  t-statistic: {t_stat_hr:.3f}")
print(f"  p-value (one-tailed): {p_value_t_hr_one:.4f}")
print(f"  Significant (p < 0.05): {'YES' if p_value_t_hr_one < 0.05 else 'NO'}")

# ============================================
# 5. ROBUSTNESS ACROSS SAMPLING RATIOS
# ============================================
print("\n" + "-"*70)
print("5. ROBUSTNESS ANALYSIS BY SAMPLING RATIO")
print("-"*70)

for ratio in SAMPLING_RATIOS:
    ratio_results = [r for r in grid_results if r['ratio'] == ratio]
    ratio_improvements = [(r['ndcg@10'] - bce_baselines[ratio]['ndcg@10']) / bce_baselines[ratio]['ndcg@10'] * 100
                          for r in ratio_results]
    wins = sum(1 for imp in ratio_improvements if imp > 0)

    print(f"\n1:{ratio} sampling:")
    print(f"  FL wins: {wins}/{len(ratio_results)} configs")
    print(f"  Mean improvement: {np.mean(ratio_improvements):+.2f}%")
    print(f"  Best improvement: {np.max(ratio_improvements):+.2f}%")

    # Find best config
    best_idx = np.argmax([r['ndcg@10'] for r in ratio_results])
    best_config = ratio_results[best_idx]
    print(f"  Best config: gamma={best_config['gamma']}, alpha={best_config['alpha']}")

# ============================================
# 6. SUMMARY
# ============================================
print("\n" + "="*70)
print("STATISTICAL ANALYSIS SUMMARY")
print("="*70)

print(f"""
Based on {fl_total} Focal Loss configurations vs BCE baselines:

EVIDENCE FOR H1 (Focal Loss improves NeuMF):
  - Win rate: {fl_wins['ndcg@10']}/{fl_total} ({fl_wins['ndcg@10']/fl_total*100:.1f}%) on NDCG@10
  - Sign test: p = {p_value_ndcg:.4f} {'(significant)' if p_value_ndcg < 0.05 else '(not significant)'}
  - T-test: p = {p_value_t_ndcg_one:.4f} {'(significant)' if p_value_t_ndcg_one < 0.05 else '(not significant)'}
  - Effect size: {cohens_d_ndcg:.3f} ({interpret_cohens_d(cohens_d_ndcg)})

EVIDENCE FOR H2 (Robustness across sampling ratios):
  - Improvements seen at: {sum(1 for r in SAMPLING_RATIOS if np.mean([(row['ndcg@10'] - bce_baselines[r]['ndcg@10'])/bce_baselines[r]['ndcg@10']*100 for row in grid_results if row['ratio'] == r]) > 0)}/{len(SAMPLING_RATIOS)} ratios

NOTE: These results are from single-seed experiments across hyperparameter
configurations. For publication-quality claims, use the multi-seed Wilcoxon
test in the next section.
""")
print("="*70)

## Statistical Significance Testing: Wilcoxon Signed-Rank Test (A100 Optimized)

**Purpose**: Determine if the differences between BCE and Focal Loss are statistically significant.

### A100 GPU Optimizations

This cell is optimized for NVIDIA A100 GPU with the following settings:

| Setting | Default | A100 Optimized | Speedup Factor |
|---------|---------|----------------|----------------|
| Train batch size | 256 | 4096 | ~16x fewer iterations |
| Eval batch size | 4096 | 8192 | ~2x faster eval |
| Mixed precision (AMP) | Off | On | ~2x via Tensor Cores |
| cuDNN benchmark | Off | On | Optimized convolutions |
| DataLoader workers | 0 | 4 | Parallel data loading |
| Eval frequency | Every epoch | Every 3 epochs | ~3x less eval overhead |

**Expected speedup**: 3-5x faster than default settings

### Statistical Methodology

The Wilcoxon signed-rank test is a non-parametric test for paired comparisons that:
- Does not assume normality of the data
- Is appropriate for comparing the same model trained with different loss functions
- Uses paired observations (same seed, different methods)

**Methodology** (from Section 5.6 of the paper):
- Run each configuration with 10 different random seeds
- Perform Wilcoxon signed-rank test for paired comparisons
- Apply Bonferroni correction for multiple comparisons
- Report effect size (rank-biserial correlation)

**Significance Level**: p < 0.05 (after Bonferroni correction)

In [ ]:
# ============================================
# STATISTICAL SIGNIFICANCE TESTING
# Wilcoxon Signed-Rank Test - A100 OPTIMIZED
# ============================================
# NOTE: This cell runs multi-seed experiments for statistical validation.
# The Wilcoxon test requires PAIRED observations (same seed, different methods).
#
# A100 OPTIMIZATIONS APPLIED:
# - Larger batch sizes (BCE: 2048, Custom losses: 1024)
# - cuDNN benchmark mode enabled
# - Increased DataLoader workers
# - Reduced evaluation frequency
#
# NOTE: AMP disabled and smaller batch for custom loss functions (FocalLoss/AlphaBCE)
# to ensure numerical stability with torch.log() operations.
#
# Expected speedup: 2-3x faster than default settings
# ============================================
from scipy import stats
import time

# ============================================
# A100 GPU OPTIMIZATIONS
# ============================================
def get_a100_optimized_config(base_config, seed, use_amp=True, batch_size=2048):
    """
    Apply A100-specific optimizations to config.

    Optimizations:
    - train_batch_size: Configurable (2048 for BCE, 1024 for custom losses)
    - eval_batch_size: 8192 (2x increase)
    - enable_amp: Only for standard BCE (disabled for custom losses)
    - worker: 4 (parallel data loading)
    - eval_step: 3 (evaluate every 3 epochs instead of 1)
    - show_progress: False (reduce I/O overhead)
    """
    config = base_config.copy()
    config.update({
        'seed': seed,
        # Batch size optimizations
        'train_batch_size': batch_size,
        'eval_batch_size': 8192,
        # Mixed precision - only for standard loss functions
        'enable_amp': use_amp,
        # DataLoader optimization
        'worker': 4,
        # Reduce evaluation overhead
        'eval_step': 3,
        # Reduce logging overhead
        'show_progress': False,
    })
    return config

# Enable cuDNN autotuning for optimal convolution algorithms
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    # Check if A100 is available
    gpu_name = torch.cuda.get_device_name(0)
    is_a100 = 'A100' in gpu_name
    print(f"GPU: {gpu_name}")
    print(f"A100 detected: {is_a100}")
    print(f"cuDNN benchmark: ENABLED")
    if is_a100:
        print("Using A100-optimized settings")
    else:
        print("NOTE: Not an A100, but optimizations will still help")
else:
    is_a100 = False
    print("WARNING: No GPU detected, running on CPU (will be slow)")

# ============================================
# CONFIGURATION
# ============================================
REUSE_RESULTS = False  # Set True to skip training and use pre-computed results
TEST_RATIO = 50        # Primary sampling ratio (1:50)
NUM_SEEDS = 10
SEEDS = list(range(42, 42 + NUM_SEEDS))

# Batch sizes: larger for BCE (stable), smaller for custom losses (numerical safety)
BCE_BATCH_SIZE = 2048      # 8x default
CUSTOM_BATCH_SIZE = 1024   # 4x default (more conservative for custom losses)

if REUSE_RESULTS:
    # ============================================
    # OPTION 1: Use pre-computed multi-seed results
    # Replace these with your actual 10-seed results
    # ============================================
    bce_scores = {
        'ndcg@10': [0.054, 0.052, 0.055, 0.053, 0.056, 0.051, 0.054, 0.055, 0.053, 0.052],
        'hit@10':  [0.112, 0.108, 0.115, 0.110, 0.118, 0.105, 0.112, 0.114, 0.109, 0.107]
    }
    focal_scores = {
        'ndcg@10': [0.059, 0.058, 0.061, 0.057, 0.062, 0.056, 0.060, 0.059, 0.058, 0.057],
        'hit@10':  [0.125, 0.122, 0.130, 0.120, 0.132, 0.118, 0.127, 0.124, 0.121, 0.119]
    }
    alpha_bce_scores = {
        'ndcg@10': [0.058, 0.056, 0.059, 0.055, 0.060, 0.054, 0.058, 0.057, 0.056, 0.055],
        'hit@10':  [0.120, 0.116, 0.124, 0.114, 0.126, 0.112, 0.121, 0.118, 0.115, 0.113]
    }
    print(f"\nUsing PRE-COMPUTED results from {NUM_SEEDS} seeds")
    print("(Set REUSE_RESULTS = False to run fresh experiments)")

else:
    # ============================================
    # OPTION 2: Run fresh multi-seed experiments (A100 OPTIMIZED)
    # ============================================
    print("\n" + "="*70)
    print("WILCOXON SIGNED-RANK TEST: Statistical Significance (A100 OPTIMIZED)")
    print("="*70)
    print(f"Running {NUM_SEEDS} seeds per configuration...")
    print(f"Seeds: {SEEDS}")
    print(f"Sampling ratio: 1:{TEST_RATIO}")
    print(f"Total models to train: {NUM_SEEDS * 3}")
    print()
    print("A100 Optimizations:")
    print(f"  - BCE batch size: {BCE_BATCH_SIZE} (vs default 256)")
    print(f"  - Custom loss batch size: {CUSTOM_BATCH_SIZE} (conservative)")
    print(f"  - Mixed precision (AMP): BCE only")
    print(f"  - cuDNN benchmark: Enabled")
    print(f"  - DataLoader workers: 4")
    print(f"  - Eval frequency: Every 3 epochs")
    print("="*70)

    # Reload focal_loss_utils to get updated loss functions
    import importlib
    import focal_loss_utils
    importlib.reload(focal_loss_utils)
    from focal_loss_utils import train_neumf_focal_loss, train_neumf_alpha_bce
    print("Reloaded focal_loss_utils with improved numerical stability")

    bce_scores = {'ndcg@10': [], 'hit@10': []}
    focal_scores = {'ndcg@10': [], 'hit@10': []}
    alpha_bce_scores = {'ndcg@10': [], 'hit@10': []}

    total_start_time = time.time()

    for i, seed in enumerate(SEEDS):
        seed_start_time = time.time()
        print(f"\n--- Seed {seed} ({i+1}/{NUM_SEEDS}) ---")

        # Train BCE (can use AMP and larger batch safely)
        print("  Training BCE...", end=" ", flush=True)
        t0 = time.time()
        bce_config = get_a100_optimized_config(
            configs[TEST_RATIO], seed, use_amp=True, batch_size=BCE_BATCH_SIZE
        )
        bce_config['loss_type'] = 'BCE'
        result_bce_seed = run_recbole(
            model='NeuMF',
            dataset=DATASET,
            config_dict=bce_config
        )
        bce_scores['ndcg@10'].append(result_bce_seed['test_result'].get('ndcg@10', 0))
        bce_scores['hit@10'].append(result_bce_seed['test_result'].get('hit@10', 0))
        print(f"done ({time.time()-t0:.1f}s)")

        # Train Focal Loss (disable AMP, use smaller batch for stability)
        print("  Training Focal Loss...", end=" ", flush=True)
        t0 = time.time()
        fl_config = get_a100_optimized_config(
            configs[TEST_RATIO], seed, use_amp=False, batch_size=CUSTOM_BATCH_SIZE
        )
        if 'loss_type' in fl_config:
            del fl_config['loss_type']
        result_fl_seed = train_neumf_focal_loss(
            config_dict=fl_config,
            dataset=DATASET,
            gamma=GAMMA,
            alpha=ALPHA,
            seed=seed,
            track_dynamics=False
        )
        focal_scores['ndcg@10'].append(result_fl_seed['test_result'].get('ndcg@10', 0))
        focal_scores['hit@10'].append(result_fl_seed['test_result'].get('hit@10', 0))
        print(f"done ({time.time()-t0:.1f}s)")

        # Train Alpha-BCE (disable AMP, use smaller batch for stability)
        print("  Training Alpha-BCE...", end=" ", flush=True)
        t0 = time.time()
        alpha_config = get_a100_optimized_config(
            configs[TEST_RATIO], seed, use_amp=False, batch_size=CUSTOM_BATCH_SIZE
        )
        if 'loss_type' in alpha_config:
            del alpha_config['loss_type']
        result_alpha_seed = train_neumf_alpha_bce(
            config_dict=alpha_config,
            dataset=DATASET,
            alpha=ALPHA,
            seed=seed,
            track_dynamics=False
        )
        alpha_bce_scores['ndcg@10'].append(result_alpha_seed['test_result'].get('ndcg@10', 0))
        alpha_bce_scores['hit@10'].append(result_alpha_seed['test_result'].get('hit@10', 0))
        print(f"done ({time.time()-t0:.1f}s)")

        seed_time = time.time() - seed_start_time
        print(f"  Results - BCE: {bce_scores['ndcg@10'][-1]:.4f}, FL: {focal_scores['ndcg@10'][-1]:.4f}, Alpha-BCE: {alpha_bce_scores['ndcg@10'][-1]:.4f}")
        print(f"  Seed {seed} completed in {seed_time:.1f}s")

        # Estimate remaining time
        elapsed = time.time() - total_start_time
        avg_per_seed = elapsed / (i + 1)
        remaining = avg_per_seed * (NUM_SEEDS - i - 1)
        print(f"  Estimated time remaining: {remaining/60:.1f} min")

    total_time = time.time() - total_start_time
    print("\n" + "="*70)
    print(f"Multi-seed training complete!")
    print(f"Total time: {total_time/60:.1f} minutes ({total_time:.0f}s)")
    print(f"Average per seed: {total_time/NUM_SEEDS:.1f}s")
    print("="*70)

In [ ]:
# ============================================
# WILCOXON TEST RESULTS
# ============================================

def compute_rank_biserial(x, y):
    """
    Compute rank-biserial correlation as effect size for Wilcoxon test.
    r = (W+ - W-) / (n*(n+1)/2)
    where W+ and W- are sums of positive and negative ranks.
    """
    n = len(x)
    diff = np.array(x) - np.array(y)
    # Remove zero differences
    diff = diff[diff != 0]
    if len(diff) == 0:
        return 0.0

    # Rank absolute differences
    abs_diff = np.abs(diff)
    ranks = stats.rankdata(abs_diff)

    # Sum of positive and negative ranks
    pos_ranks = np.sum(ranks[diff > 0])
    neg_ranks = np.sum(ranks[diff < 0])

    # Rank-biserial correlation
    n_nonzero = len(diff)
    r = (pos_ranks - neg_ranks) / (n_nonzero * (n_nonzero + 1) / 2)
    return r

def interpret_effect_size(r):
    """Interpret rank-biserial correlation effect size."""
    r_abs = abs(r)
    if r_abs < 0.1:
        return "negligible"
    elif r_abs < 0.3:
        return "small"
    elif r_abs < 0.5:
        return "medium"
    else:
        return "large"

print("="*70)
print("WILCOXON SIGNED-RANK TEST RESULTS")
print("="*70)

# Number of comparisons for Bonferroni correction
NUM_COMPARISONS = 4  # 2 metrics x 2 comparisons (FL vs BCE, FL vs Alpha-BCE)
ALPHA_LEVEL = 0.05
CORRECTED_ALPHA = ALPHA_LEVEL / NUM_COMPARISONS

print(f"\nSignificance level: {ALPHA_LEVEL}")
print(f"Bonferroni-corrected alpha: {CORRECTED_ALPHA:.4f} (for {NUM_COMPARISONS} comparisons)")

# Store all test results
test_results = []

# Test 1: Focal Loss vs BCE
print("\n" + "-"*70)
print("H1: Focal Loss vs BCE")
print("-"*70)

for metric in ['ndcg@10', 'hit@10']:
    bce = np.array(bce_scores[metric])
    fl = np.array(focal_scores[metric])

    # Wilcoxon signed-rank test (two-sided)
    statistic, p_value = stats.wilcoxon(fl, bce, alternative='two-sided')

    # Effect size
    effect_size = compute_rank_biserial(fl, bce)
    effect_interp = interpret_effect_size(effect_size)

    # Mean and std
    bce_mean, bce_std = np.mean(bce), np.std(bce)
    fl_mean, fl_std = np.mean(fl), np.std(fl)
    pct_change = (fl_mean - bce_mean) / bce_mean * 100

    # Significance
    is_significant = p_value < CORRECTED_ALPHA

    print(f"\n{metric.upper()}:")
    print(f"  BCE:        {bce_mean:.4f} +/- {bce_std:.4f}")
    print(f"  Focal Loss: {fl_mean:.4f} +/- {fl_std:.4f}")
    print(f"  Change:     {pct_change:+.2f}%")
    print(f"  Wilcoxon W: {statistic:.1f}")
    print(f"  p-value:    {p_value:.4f} {'*' if is_significant else ''}")
    print(f"  Effect size (r): {effect_size:+.3f} ({effect_interp})")
    print(f"  Significant (p < {CORRECTED_ALPHA:.4f}): {'YES' if is_significant else 'NO'}")

    test_results.append({
        'comparison': 'FL vs BCE',
        'metric': metric,
        'baseline_mean': bce_mean,
        'baseline_std': bce_std,
        'test_mean': fl_mean,
        'test_std': fl_std,
        'pct_change': pct_change,
        'wilcoxon_W': statistic,
        'p_value': p_value,
        'effect_size': effect_size,
        'significant': is_significant
    })

# Test 2: Focal Loss vs Alpha-BCE
print("\n" + "-"*70)
print("H3: Focal Loss vs Alpha-BCE (Mechanism)")
print("-"*70)

for metric in ['ndcg@10', 'hit@10']:
    alpha = np.array(alpha_bce_scores[metric])
    fl = np.array(focal_scores[metric])

    # Wilcoxon signed-rank test (two-sided)
    statistic, p_value = stats.wilcoxon(fl, alpha, alternative='two-sided')

    # Effect size
    effect_size = compute_rank_biserial(fl, alpha)
    effect_interp = interpret_effect_size(effect_size)

    # Mean and std
    alpha_mean, alpha_std = np.mean(alpha), np.std(alpha)
    fl_mean, fl_std = np.mean(fl), np.std(fl)
    pct_change = (fl_mean - alpha_mean) / alpha_mean * 100

    # Significance
    is_significant = p_value < CORRECTED_ALPHA

    print(f"\n{metric.upper()}:")
    print(f"  Alpha-BCE:  {alpha_mean:.4f} +/- {alpha_std:.4f}")
    print(f"  Focal Loss: {fl_mean:.4f} +/- {fl_std:.4f}")
    print(f"  Change:     {pct_change:+.2f}%")
    print(f"  Wilcoxon W: {statistic:.1f}")
    print(f"  p-value:    {p_value:.4f} {'*' if is_significant else ''}")
    print(f"  Effect size (r): {effect_size:+.3f} ({effect_interp})")
    print(f"  Significant (p < {CORRECTED_ALPHA:.4f}): {'YES' if is_significant else 'NO'}")

    test_results.append({
        'comparison': 'FL vs Alpha-BCE',
        'metric': metric,
        'baseline_mean': alpha_mean,
        'baseline_std': alpha_std,
        'test_mean': fl_mean,
        'test_std': fl_std,
        'pct_change': pct_change,
        'wilcoxon_W': statistic,
        'p_value': p_value,
        'effect_size': effect_size,
        'significant': is_significant
    })

In [ ]:
# ============================================
# SUMMARY TABLE
# ============================================
print("\n" + "="*70)
print("STATISTICAL SIGNIFICANCE SUMMARY TABLE")
print("="*70)

summary_stats = []
for r in test_results:
    summary_stats.append({
        'Comparison': r['comparison'],
        'Metric': r['metric'].upper(),
        'Baseline': f"{r['baseline_mean']:.4f}+/-{r['baseline_std']:.4f}",
        'Focal Loss': f"{r['test_mean']:.4f}+/-{r['test_std']:.4f}",
        'Change': f"{r['pct_change']:+.2f}%",
        'p-value': f"{r['p_value']:.4f}",
        'Effect (r)': f"{r['effect_size']:+.3f}",
        'Sig?': 'Yes*' if r['significant'] else 'No'
    })

stats_df = pd.DataFrame(summary_stats)
print("\n" + stats_df.to_string(index=False))

print(f"\n* Significant at Bonferroni-corrected alpha = {CORRECTED_ALPHA:.4f}")

# Final interpretation
print("\n" + "="*70)
print("STATISTICAL CONCLUSIONS")
print("="*70)

h1_sig = any(r['significant'] and r['comparison'] == 'FL vs BCE' and r['pct_change'] > 0
             for r in test_results)
h3_sig = any(r['significant'] and r['comparison'] == 'FL vs Alpha-BCE' and r['pct_change'] > 0
             for r in test_results)

print("\nH1 (Focal Loss vs BCE):")
if h1_sig:
    print("  STATISTICALLY SIGNIFICANT improvement detected")
else:
    sig_results = [r for r in test_results if r['comparison'] == 'FL vs BCE']
    if all(r['pct_change'] < 0 for r in sig_results):
        print("  No significant improvement; Focal Loss may underperform BCE")
    else:
        print("  No statistically significant difference detected")

print("\nH3 (Focal Loss vs Alpha-BCE - Mechanism):")
if h3_sig:
    print("  STATISTICALLY SIGNIFICANT: Focusing effect (gamma > 0) provides benefit")
else:
    print("  NOT SIGNIFICANT: Class weighting alone (Alpha-BCE) may be sufficient")

print("\n" + "="*70)
print("Note: Effect size interpretation (rank-biserial r):")
print("  |r| < 0.1: negligible, 0.1-0.3: small, 0.3-0.5: medium, > 0.5: large")
print("="*70)

## Paired Results Visualization

In [ ]:
# ============================================
# PAIRED RESULTS VISUALIZATION
# ============================================
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: NDCG@10 across seeds
ax1 = axes[0]
seeds_idx = range(1, NUM_SEEDS + 1)
ax1.plot(seeds_idx, bce_scores['ndcg@10'], 'o-', label='BCE', color='blue', markersize=8)
ax1.plot(seeds_idx, focal_scores['ndcg@10'], 's-', label='Focal Loss', color='red', markersize=8)
ax1.plot(seeds_idx, alpha_bce_scores['ndcg@10'], '^-', label='Alpha-BCE', color='green', markersize=8)
ax1.set_xlabel('Seed Index', fontsize=12)
ax1.set_ylabel('NDCG@10', fontsize=12)
ax1.set_title('NDCG@10 Across Random Seeds', fontsize=14)
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)
ax1.set_xticks(seeds_idx)

# Plot 2: HR@10 across seeds
ax2 = axes[1]
ax2.plot(seeds_idx, bce_scores['hit@10'], 'o-', label='BCE', color='blue', markersize=8)
ax2.plot(seeds_idx, focal_scores['hit@10'], 's-', label='Focal Loss', color='red', markersize=8)
ax2.plot(seeds_idx, alpha_bce_scores['hit@10'], '^-', label='Alpha-BCE', color='green', markersize=8)
ax2.set_xlabel('Seed Index', fontsize=12)
ax2.set_ylabel('HR@10', fontsize=12)
ax2.set_title('HR@10 Across Random Seeds', fontsize=14)
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)
ax2.set_xticks(seeds_idx)

plt.tight_layout()
plt.savefig('wilcoxon_paired_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved as 'wilcoxon_paired_results.png'")

In [ ]:
# ============================================
# BOX PLOT COMPARISON
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# NDCG@10 box plot
ax1 = axes[0]
data_ndcg = [bce_scores['ndcg@10'], alpha_bce_scores['ndcg@10'], focal_scores['ndcg@10']]
bp1 = ax1.boxplot(data_ndcg, labels=['BCE', 'Alpha-BCE', 'Focal Loss'], patch_artist=True)
colors = ['#3498db', '#2ecc71', '#e74c3c']
for patch, color in zip(bp1['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax1.set_ylabel('NDCG@10', fontsize=12)
ax1.set_title('NDCG@10 Distribution (10 seeds)', fontsize=14)
ax1.grid(True, alpha=0.3, axis='y')

# HR@10 box plot
ax2 = axes[1]
data_hr = [bce_scores['hit@10'], alpha_bce_scores['hit@10'], focal_scores['hit@10']]
bp2 = ax2.boxplot(data_hr, labels=['BCE', 'Alpha-BCE', 'Focal Loss'], patch_artist=True)
for patch, color in zip(bp2['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax2.set_ylabel('HR@10', fontsize=12)
ax2.set_title('HR@10 Distribution (10 seeds)', fontsize=14)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('wilcoxon_boxplot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved as 'wilcoxon_boxplot.png'")